In [1]:
!pip install opencv-python

In [2]:
from google.colab import drive
import sys
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import openpyxl
from sklearn.preprocessing import StandardScaler
import joblib

In [3]:
import sys
sys.executable
sys.path

['/content',
 '/env/python',
 '/usr/lib/python311.zip',
 '/usr/lib/python3.11',
 '/usr/lib/python3.11/lib-dynload',
 '',
 '/usr/local/lib/python3.11/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.11/dist-packages/IPython/extensions',
 '/usr/local/lib/python3.11/dist-packages/setuptools/_vendor',
 '/root/.ipython']

In [4]:
drive.mount('/content/drive')

#Define la parte del directorio que quieres trabajar
path = "/content/drive/MyDrive/Mod2/Finanzas"

sys.path.append(path)##para importar archivo de funciones propias a traves de import

Mounted at /content/drive


In [25]:
if __name__=="__main__":

    #### cargar datos ####
    # Cargar los datos históricos y de nuevos créditos
    ruta_historicos = '/content/drive/MyDrive/Mod2/Finanzas/data/datos_historicos.csv'
    df_historicos = pd.read_csv(ruta_historicos)

    ruta_nuevos = '/content/drive/MyDrive/Mod2/Finanzas/data/datos_nuevos_creditos.csv'
    df_nuevos_creditos = pd.read_csv(ruta_nuevos)

    # Mantener una copia de los datos originales de nuevos créditos para fusionar más tarde
    df_nuevos_creditos_original = df_nuevos_creditos.copy()

    #### Preprocesamiento de datos históricos ####
    # Eliminar las variables 'ID', 'MaritalStatus', 'YearsAtCurrentAddress' de datos_historicos ya que no se usaron en entrenamiento
    df_historicos = df_historicos.drop(columns=['ID', 'MaritalStatus', 'YearsAtCurrentAddress'], errors='ignore')

    # Aplicar dummies a las variables categóricas de datos_historicos
    X_categoricas_historicos = df_historicos.select_dtypes(include='object')
    df_historicos = pd.get_dummies(df_historicos, columns=X_categoricas_historicos.columns, drop_first=True)

    # Separar la variable objetivo para escalar solo las características numéricas
    X_historicos_features = df_historicos.drop(columns=['NoPaidPerc'], errors='ignore')
    y_historicos = df_historicos['NoPaidPerc']

    # Seleccionar columnas numéricas para escalar en datos_historicos
    X_numericas_historicos = X_historicos_features.select_dtypes(include=['int64', 'float64'])
    cols_for_scaling_historicos = X_numericas_historicos.columns

    # Ajustar el escalador con los datos históricos
    scaler = StandardScaler()
    X_historicos_features[cols_for_scaling_historicos] = scaler.fit_transform(X_numericas_historicos)


    #### Preprocesamiento de nuevos datos de crédito (aplicando las mismas transformaciones) ####
    # Eliminar las variables 'ID', 'MaritalStatus', 'YearsAtCurrentAddress' de datos_nuevos_creditos ya que no se usaron en entrenamiento, Se borra NewLoanApplication porque no esta datos_historicos
    df_nuevos_creditos = df_nuevos_creditos.drop(columns=['ID', 'MaritalStatus', 'YearsAtCurrentAddress', 'NewLoanApplication'], errors='ignore')

    # Aplicar dummies a las variables categóricas de datos_nuevos_creditos
    X_categoricas_nuevos = df_nuevos_creditos.select_dtypes(include='object')
    df_nuevos_creditos= pd.get_dummies(df_nuevos_creditos, columns=X_categoricas_nuevos.columns, drop_first=True)


    # Aplicar el mismo escalado a las características numéricas de los nuevos datos de crédito
    # Seleccionar columnas numéricas para escalar en df_nuevos_creditos_processed
    X_numericas_nuevos= df_nuevos_creditos.select_dtypes(include=['int64', 'float64'])
    cols_for_scaling_nuevos = X_numericas_nuevos.columns
    df_nuevos_creditos[cols_for_scaling_nuevos] = scaler.transform(X_numericas_nuevos)


    #Cargar modelo
    modelo=joblib.load('/content/drive/MyDrive/Mod2/Finanzas/salidas/model.pkl')

    # Realizar predicciones sobre los nuevos datos de crédito
    preds_nuevos = modelo.predict(df_nuevos_creditos)

    # Realizar predicciones sobre los historicos
    preds_historicos = modelo.predict(X_historicos_features)
    preds_historicos=pd.DataFrame(preds_historicos, columns=['NoPaidPerc_Predicho'])

    # Crear un DataFrame con las predicciones y los IDs correspondientes de los nuevos datos de crédito
    preds_df_nuevos = pd.DataFrame(preds_nuevos, columns=['NoPaidPerc_Predicho'])
    preds_df_nuevos['ID'] = df_nuevos_creditos_original['ID']

    # Calcular la tasa de riesgo y la tasa de interés final para los nuevos datos de crédito
    preds_df_nuevos['int_rc'] = preds_df_nuevos["NoPaidPerc_Predicho"] / (1 - preds_df_nuevos["NoPaidPerc_Predicho"])
    limite_99=preds_df_nuevos['int_rc'].quantile(0.99)
    limite_6=preds_df_nuevos['int_rc'].quantile(0.06)
    preds_df_nuevos['int_rc'] = preds_df_nuevos['int_rc'].clip(lower=limite_6, upper=limite_99)

    # Fusionar la tasa de riesgo calculada con el DataFrame original de nuevos datos de crédito
    df_nuevos_creditos_final = pd.merge(df_nuevos_creditos_original, preds_df_nuevos[['ID', 'int_rc']], on='ID', how='left')

    # Mostrar solo las columnas 'ID' e 'Tasa_riesgo'
    display(df_nuevos_creditos_final[['ID', 'int_rc']])

    # Define el directorio de salida
    salida = '/content/drive/MyDrive/Mod2/Finanzas'

    # Guarda solo las columnas 'ID' e 'Tasa_riesgo' en un archivo de Excel
    tasa_riesgo = df_nuevos_creditos_final[['ID', 'int_rc']].copy()
    tasa_riesgo.to_csv(os.path.join(salida, 'grupo_3.csv'), index=False)

,ID,int_rc
0,10001,0.271831
1,10002,0.201030
2,10003,0.227954
3,10004,0.191719
4,10005,0.108517
...,...,...
1053,11054,0.256899
1054,11055,0.287853
1055,11056,0.220216
1056,11057,0.188962


In [26]:
tasa_riesgo.describe()

,ID,int_rc
count,1058.000000,1058.000000
mean,10529.500000,0.198817
std,305.562596,0.053866
min,10001.000000,0.108517
25%,10265.250000,0.157623
50%,10529.500000,0.197037
75%,10793.750000,0.241822
max,11058.000000,0.305505


In [27]:
preds_historicos.describe()

,NoPaidPerc_Predicho
count,10000.000000
mean,0.226022
std,0.043986
min,0.022515
25%,0.200717
50%,0.230755
75%,0.256511
max,0.355568
